In [19]:
import re
import string
import pandas as pd

## Metin Ön İşleme

In [20]:
df = pd.read_csv("fenerbahce_beko_news_all.csv", encoding="utf-8-sig")
print(df.shape)
df.head()

(10497, 3)


,title,text,date
0,Bogdanovic 62 Maç Sonra İlk Kez,By Eurohoops Team /info@eurohoops.net\nFenerba...,20/May/16 18:34
1,Fenerbahçe Deplasmanda İşi Bitirdi,By Semih Tuna/info@eurohoops.net\nSpor Toto Ba...,20/May/16 18:25
2,2016 Final Four Filmi,By Eurohoops team/info@eurohoops.net\nBerlin’d...,20/May/16 14:24
3,"Kenan Sipahi Karşıyaka’yla Parlıyor, Geleceği ...",By Bugra Uzar/info@eurohoops.net\nBu sezon baş...,19/May/16 14:28
4,Fenerbahçe İlk Adımı Attı,By Eurohoops team/info@eurohoops.net\nSpor Tot...,18/May/16 22:09


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10497 entries, 0 to 10496
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   10497 non-null  object
 1   text    10416 non-null  object
 2   date    10497 non-null  object
dtypes: object(3)
memory usage: 246.2+ KB


In [22]:
df.isnull().sum()

title     0
text     81
date      0
dtype: int64

In [23]:
df = df.dropna()

In [24]:
print(df.isnull().sum())
print("Shape:",df.shape)

title    0
text     0
date     0
dtype: int64
Shape: (10416, 3)


In [25]:
df.iloc[0]

title                      Bogdanovic 62 Maç Sonra İlk Kez
text     By Eurohoops Team /info@eurohoops.net\nFenerba...
date                                       20/May/16 18:34
Name: 0, dtype: object

In [26]:
# tüm notebook için uygulanır.
# pd.set_option("display.max_colwidth", None)

# sadece satır için bu şekilde çalışır.
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[4])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [27]:
def text_clean(text):
    
    # 1) Başlangıçtaki "by X / email" kısmını kaldır
    text = re.sub(r"^by\s+.*?\n", " ", text, flags=re.IGNORECASE)

    # 2) Email adreslerini kaldır
    text = re.sub(r"\S+@\S+", " ", text)

    # 3) Sabit reklam / tekrar eden cümleleri temizle
    patterns = [
        r"eurohoops türkiye’nin instagram hesabını takip etmek için tıklayın!?",
        r"basketbol gündemindeki en son gelişmeleri kaçırmamak için tıklayın!?",
        r"euroleague gündemindeki son gelişmeleri kaçırmamak için tıklayın!?",
    ]
    for p in patterns:
        text = re.sub(p, " ", text, flags=re.IGNORECASE)

    # 4) Satır sonlarını boşlukla değiştir
    text = text.replace("\n", " ")

    # 5) Küçük harfe çevir
    text = text.lower()

    # 6) Harf, sayı ve noktalama dışındaki karakterleri temizle
    allowed = string.punctuation + "çğıöşüÇĞİÖŞÜ"
    text = re.sub(rf"[^a-zA-Z0-9{re.escape(allowed)}\s]", " ", text)

    # 7) Tekrarlayan noktalama işaretlerini sadeleştir
    text = re.sub(r"([!?.,:;])\1+", r"\1", text)

    # 8) Fazla boşlukları temizle
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [28]:
df["clean_text"] = df["text"].apply(text_clean)

In [29]:
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[4])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [30]:
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[15])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Dixon: “Galibiyetten Bir Ribaunt Uzaklıktaydık”
text          Berlin’den Buğra Uzar Bildirdi /info@eurohoops.net\nFenerbahçeoyun kurucusuBobby Dixon, maç sonu Eurohoops’a açıklamalarda bulundu.\nDixon:\n“CSKAsert bir takım. Biz geri gelmek için elimizden geleni yaptık. Bir ribaunt uzaklıktaydık adamım… Galibiyetten yalnızca bir ribaunt uzaklıktaydık… Eğer o ribauntu alsaydık maç uzatmaya gitmeyecek ve kazanacaktık. Maçı kazandılar… Takımımla gurur duyuy

In [31]:
df.to_csv(
    "fenerbahce_beko_news_all_cleaned.csv",
    index=False,
    encoding="utf-8-sig"
)